In [10]:
# Please see below this block for a demonstration

############## Main Functions ##############

# Given D such that K = QQ(sqrt(-D)) is a CM field and X > 0, returns a list of all primes p < X such that
# there exists an elliptic curve with CM by O_K for which the reduced curve modulo some place v | p of K has
# order divisible by p. These primes necessarily split in K/QQ.
def valid_primes(D, X):
    P = Primes()
    p = P.next(1)
    valid = []
    while p < X:
        if len(valid_residues(D, p)) > 0:
            valid.append(p)
        p = P.next(p)
    return valid


# Given D such that K = QQ(sqrt(-D)) is a CM field and a prime p splitting in K,
# returns the list of a0 in GF(p) such that at any place v | p of K, the elliptic curve E: y^2 = f_{D,a} in 
# [Joux-Morain, Tableau 1] has special fiber of order p whenever a = a0 (mod p) in O_{K_v} 
def valid_residues(D, p):
    A, B = _curve_coeffs(D)
    a0_vals = []
    for a0 in range(0,p):
        try:
            E_red = EllipticCurve(GF(p), [A(a0), B(a0)])
            if E_red.order() % p == 0:
                a0_vals.append(a0)
        except:
            pass
        
    return a0_vals


# Given D such that K = QQ(sqrt(-D)) is a CM field and a prime p as in the description of valid_primes above,
# returns a factorization p = pi * pi_bar in O_K, together with the corresponding kernel polynomials. The kernel
# polynomials are given as weighted homogeneous polynomials in 'x' and 'a', where 'a' is the parameter used to
# define the family of elliptic curves with CM by O_K as given in [Joux-Morain, Tableau 1]
def pi_kernel_polys(D,p):
    # First, we set up a single elliptic curve having the desired CM
    RQ.<t> = QQ[]
    K.<root_D> = QQ.extension(t^2 + D)
    A,B = _curve_coeffs(D)
    E = EllipticCurve(K, [A(1), B(1)])
    
    # We then factor the pth division polynomial phi_p of E over K, and factor p = pi * pi_bar in O_K
    phi_p = E.division_polynomial(p)
    phi_p_factors = phi_p.factor()
    assert phi_p_factors.unit() == p
    assert len(phi_p_factors) == 3
    phi_1 = phi_p_factors[0][0]; phi_2 = phi_p_factors[1][0]
    
    OK = OK = K.ring_of_integers()
    p_factors = OK(p).factor()
    pi_1 = p_factors[0][0]; pi_2 = p_factors[1][0]
    
    # phi_p always has leading coefficient p, and correctly pairing the factors of p with those of phi_p
    # gives us a factorization of phi_p over O_K
    try:
        phi_1 = (pi_1 * phi_1).change_ring(OK)
        phi_2 = (pi_2 * phi_2).change_ring(OK)
    except:
        phi_1 = (pi_2 * phi_1).change_ring(OK)
        phi_2 = (pi_1 * phi_2).change_ring(OK)
    
    # Homogenize the kernel polynomials of pi and pi_bar to get the generic kernel polynomials
    wt = 1
    if D == 1:
        wt = 2
    elif D == 3:
        wt = 3
    phi_1 = _weighted_homogenize(phi_1, wt)
    phi_2 = _weighted_homogenize(phi_2, wt)
    
    # Check to see which kernel polynomial corresponds to which factor of p; phi_pi_bar has roots after localizing
    # at pi, but phi_pi only acquires roots after a degree p-1 totally ramified extension
    a0 = valid_residues(D,p)[0]
    
    v_1 = OK.valuation(OK.ideal(pi_1))
    roots_1 = []
    for b0 in range(0,p):
        if v_1(phi_2(b0,a0)) > 0:
            roots_1.append(b0)
    if len(roots_1) > 0:
        return (pi_1, phi_1), (pi_2, phi_2)
    
    roots_2 = []
    for b0 in range(0,p):
        if v_1(phi_1(b0,a0)) > 0:
            roots_2.append(b0)
    assert len(roots_2) > 0
    return (pi_1, phi_2), (pi_2, phi_1)

    
# Given the Q-factor of the p'th division polynomial of E corresponding to the pi and pi_bar torsion as a
# polynomial in x and a, and given a0 such that E has special fiber of order p whenever a = a0 (mod p),
# returns the list of tuples (b0, b1, b1a) such that the pi-torsion of E is given by {b0 + (b1 + b1a*a1)*p + O(p^2)}
def compute_pi_torsion_x_coords(pi, phi_pi_bar, a0):
    OK = pi.parent()
    v = OK.valuation(pi)
    p = pi.norm()
    
    x,a = phi_pi_bar.parent().gens()
    phi_pi_bar_x = phi_pi_bar.derivative(x)
    phi_pi_bar_a = phi_pi_bar.derivative(a)
    
    torsion = []
    for b0 in range(0,p):
        if v(phi_pi_bar(b0, a0)) > 0:
            b1 = 0
            while b1 < p:
                if v(p * phi_pi_bar_x(b0, a0) * b1 + phi_pi_bar(b0, a0)) > 1:
                    break
                b1 += 1
            
            b1a = 0
            while b1a < p:
                if v(phi_pi_bar_x(b0, a0) * b1a + phi_pi_bar_a(b0, a0)) > 0:
                    break
                b1a += 1
            
            torsion.append((b0, b1, b1a))
    return torsion


# Given D such that K = QQ(sqrt(-D)) is a CM field and a prime p, returns the result of compute_pi_torsion_x_coords
# for every residue of a such that E_a has order divisible by p
def compute_torsion(D, p):
    a0_vals = valid_residues(D,p)
    pi_data, pi_bar_data = pi_kernel_polys(D,p)
    torsion = []
    for a0 in a0_vals:
        T = compute_pi_torsion_x_coords(pi_data[0], pi_bar_data[1],a0)
        torsion.append((a0,T))
    return torsion


############## Helper Functions ##############

# Given D in {1, 2, 3, 7, 11, 19, 43, 67, 163}, returns the coefficients (A, B) as polynomials in a such that 
# E: y^2 = x^3 + Ax + B has CM by the ring of integers of Q(sqrt(-D))
def _curve_coeffs(D):
    R.<a> = ZZ[]
    if D == 1:
        return (a, R(0))
    elif D == 2:
        return (-4320*a^2, 96768*a^3)
    elif D == 3:
        return (R(0), a)
    elif D == 7:
        return (-2^4*3^4*5*7*a^2, -2^7*3^6*7^2*a^3)
    elif D == 11:
        return (-2^5*3*11*a^2, 2^4*7*11^2*a^3)
    elif D == 19:
        return (-2^3*19*a^2, 2*19^2*a^3)
    elif D == 43:
        return (-2^4*5*43*a^2, 2*3*7*43^2*a^3)
    elif D == 67:
        return (-2^3*5*11*67*a^2, 2*7*31*67^2*a^3)
    elif D == 163:
        return (-2^4*5*23*29*163*a^2, 2*7*11*19*127*163^2*a^3)
    else:
        raise ValueError(f"Q(sqrt({D})) has class number >1")

        
# Homogenizes the univariate polynomial p, with the homogenizing variable having weight wt. 
# Assumes that all monomials in p have degree divisible by wt.
def _weighted_homogenize(p, wt):
    deg = p.degree()
    R = p.parent().base_ring()
    Rhomg.<x,a> = R[]
    
    return sum([p.monomial_coefficient(m) * x^(m.degree()) * a^((deg - m.degree())/wt) for m in p.monomials()])

In [2]:
############## Example ##############
# The elliptic curve 118336.v2 in the LMFDB has short Weierstrass form:
A = -3440
B = 77658
E = EllipticCurve([A,B])

# This curve has potential CM by the ring of integers of K = QQ(sqrt(-43))
D = 43
RQ.<t> = QQ[]
K.<root_D> = QQ.extension(t^2 + D)

# Parametrizing all such elliptic curves as E_a as in [Joux-Morain, Tableau 1], we see that E = E_1
a = 1
A_gen, B_gen = _curve_coeffs(D)
assert A == A_gen(a)
assert B == B_gen(a)

# Further, this 'a' is such that when reduced modulo either place lying above p = 11 in K,
# the reduced curve has order 11:
p = 11
E_red = EllipticCurve(GF(p), [A,B])
assert E_red.order() == p

# So we may apply Theorem 4.9 to determine whether points on E give rise to a non-trivial zero-cycle when paired with
# a pi-torsion point of E, where p = pi * pi_bar. The LMFDB gives that E has rank 1, with generator as follows:
P = E([129/4, 129/8])

# Now we expand out both P_x and 'a' 11-adically to order O(p^2):
Q11 = Qp(p, prec = 2, print_pos = True)
print(Q11(P[0]))
print(Q11(a))

2 + O(11^2)
1 + O(11^2)


In [3]:
# So in our notation:
x0 = 2; x1 = 0; a0 = 1; a1 = 0

# Now factor p=11 in O_K and compute the kernel polynomials of the corresponding endomorphisms:
# (The two places will behave identically in this case, since E is defined over QQ)
pi_data, pi_bar_data = pi_kernel_polys(D,p)
pi = pi_data[0]
phi_pi_bar = pi_bar_data[1]

# A Little bit of set-up...
OK = pi.parent()
v = OK.valuation(pi)
x,a = phi_pi_bar.parent().gens()
phi_pi_bar_x = phi_pi_bar.derivative(x)
phi_pi_bar_a = phi_pi_bar.derivative(a)

# And we can check the criterion given in Theorem 4.9:
print(v(phi_pi_bar(x0, a0)/p + x1*phi_pi_bar_x(x0, a0) + a1*phi_pi_bar_a(x0, a0)))

0


In [4]:
# Since this valuation is zero, we conclude that the point P = (129/4, 129/8)
# gives rise to a global zero-cycle which is locally (at pi) non-trivial modulo p = 11 when paired as {A, P}_L/L
# with any non-zero point A in E[pi], where L = K(E[pi]); an identical statement holds for pi_bar as well.
#
# That is to say: for X = (E x E)_L, the complex
#      F^2(X) --> liminv_n F^2_AA(X)/p^n --> Hom(Br(X), QQ/ZZ)
# is exact, with the first arrow mapping {A, P}_L/L as above onto a generator for the middle term.


# It's easy to upgrade this to a pair of mod-p linearly independent symbols for L = F(E[pi]), where we want
# F/K to be a quadratic extension in which p splits. By Proposition 5.1, for any b in O_K satisfying the valuative criterion
# of Theorem 4.9,  so long as alpha = sqrt(b^3 - 3340*b + 77658) is not already a square in K and p splits in K(alpha) / K,
# the point Q = (b, alpha) is as desired.
#
# To pick our b, let's list out all of the x coordinates of pi-torsion points of our curve:
print(compute_pi_torsion_x_coords(pi, phi_pi_bar, a0))

[(0, 5, 0), (2, 4, 2), (3, 6, 3), (6, 0, 6), (10, 7, 10)]


In [5]:
# Since in this scenario a1 = 0, the p-adic expansions of b after embedding K into K_pi which we need to avoid are:
for T in compute_pi_torsion_x_coords(pi, phi_pi_bar, a0):
    print(f'{T[0]} + {T[1]}*11 + O(11^2)')

0 + 5*11 + O(11^2)
2 + 4*11 + O(11^2)
3 + 6*11 + O(11^2)
6 + 0*11 + O(11^2)
10 + 7*11 + O(11^2)


In [6]:
# So for example, take b = 2:
# - since b matches an x-coordinate of the pi_bar torsion up to order p, p splits in K(alpha)/K
# - since b doesn't match this pi_bar torsion point to any higher order, {A, Q}_L/L is non-trivial modulo p at pi
# - since b^3 - 3340*b + 77658 = 70986 is not a square in K, we have that {A, Q}_L/L is linearly independent from {A, P}_L/L as above:
RK.<s> = K[]
f = s^3 + A*s + B
assert len((s^2 - f(2)).factor()) == 1

# We may conclude that taking L = K(E[pi], sqrt(70986)) and X = (E x E)_L, the complex 
#      F^2(X) --> liminv_n F^2_AA(X)/p^n --> Hom(Br(X), QQ/ZZ)
# is exact, with the first arrow mapping {A, P}_L/L and {A, Q}_L/L onto generators for the middle term.